# Basin analysis
Code to calculate autocorrelations and cross-correlations for data within a specific basin or set of basins


In [6]:
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt
import scipy
import statsmodels.api as sm
from scipy import ndimage
from scipy import signal

In [10]:
#define an e-folding time function that takes an autocorrelation or cross-correlation vector as its input

#the function requires the values of autocorrelation or crosscorrelation and the length of the time series used
# to calculate those values

#the function returns the e-folding value, the value that fell below e, and the degrees of freedom

def e_fold(corrTS,lenTS):
    #start from the max value in the first year
    maxVal = np.max(np.abs(corrTS[:12]))
    maxValLoc = np.argmax(np.abs(corr[:12]))
    maxValwSign = corrTS[maxValLoc]
    #initialize Te, i, val_Te, and dof in case the e-folding value is never reached
    Te = np.nan
    i = np.nan
    dof = np.nan
    val_Te = np.nan
    for i,val in enumerate(np.abs(corrTS[maxValLoc:])):
        if val < maxVal/np.e: #limit the values of correlation or cross-corr to the first year for max value
            Te = i
            val_Te = val
            break
    if np.isfinite(Te): #if Te is not nan, calc the dof
        dof = (lenTS*1)/(2*(Te+1)) #account for pythonic indexing w/ Te+1
    return Te, val_Te, dof, maxValwSign, maxValLoc

In [17]:
gapFills = ['glws2']#['clim','glws2'] #gapFill as either 'clim' or 'glws2'

detrends = ['Gauss12','Gauss24','Gauss36','Gauss48','linear'] #detrend the observations using either a linear trend or a Gaussian filter

#this first variable is what indexes onto the pickle object. Because our main interest is in predicting TWS I am using only GRACE for now
variables = ['grace_lwe_cm']#['e_gleam_mm_mon','glws2_gwater_mm','grace_lwe_cm','grun_ens_runoff_mm_day',
            # 'prec_fldas_mm_day','smroot_gleam_mm3_mm3','smsurf_gleam_mm3_mm3','lai_gimms4g_m2_m2']
    
variable2s = ['glws2_gwater_mm','grace_lwe_cm','smroot_gleam_mm3_mm3','lai_gimms4g_m2_m2']


basins = ['ABHE BAD (also ABHE BID HAYK)','ACARAU','AISEN','ALAKOL','ALAZEYA',
 'ALBANY','ALSEK','ALTAMAHA','AMACURO (also AMAKURA)',
 'AMAZON (also AMAZONAS)','AMUR','ANABAR','ANADYR','ANDERSON',
 'ANGERMANALVEN','APALACHICOLA','ARAGUARI','ARAL SEA','ARMERIA','ARNAUD',
 'ASHBURTON','ATOUI','ATRATO','ATREK','ATTAWAPISKAT','AZOPARDO','BACK',
 'BAKER','BALEINE, GRANDE RIVIERE DE LA','BALEINE, RIVIERE DE LA',
 'BANCANNIA LAKE','BANDAMA','BARAKA','BARIMA','BENGAWAN SOLO','BIA',
 'BIOBIO','BLACKWOOD','BLANCA','BRAHMANI','BRAHMAPUTRA','BRANTAS','BRAZOS',
 'BULLER','BULUNGAN','BURDEKIN','BUZI (also BUDZI)','CANCOSA','CANDELARIA',
 'CANETE','CANIAPISCAU','CAPE FEAR','CAPIM','CATATUMBO','CAUVERY',
 'CAVALLY','CESTOS (also NUON)','CHAD','CHANGUINOLA','CHAO PHRAYA',
 'CHELIF','CHERGUI','CHEW BAHIR (also LAKE STEFANIE)','CHICO',
 'CHILOANGO (also SHILOANGO, also LOUANGO)','CHIRA','CHOLUTECA','CHUBUT',
 'CHURCHILL','CHUY (also SHU)','CISNES','CLUTHA','COATAN','COCO','COIG',
 'COLORADO','COLORADO (also COLORADO RIVER)','COLUMBIA','COLVILLE',
 'COMOE (also KOMOE)','CONCEPCION','CONGO','CONNECTICUT','COPIAPO',
 'COPPENAME RIVIER','COPPER','COPPERMINE','CORUBAL (also KOLIBA)',
 'COSAPILLA','COURANTYNE','CROSS','CUANZA','CUVELAI','CUYUNI','DALING HE',
 'DALY','DAMODAR','DANUBE','DAOURA','DASHT KAUR',
 'DAUGAVA (also ZAPADNAYA DVINA, also ZAKHODNAYA DZVINA)','DE CONTAS',
 'DE GREY RIVER','DEAD SEA','DELAWARE','DESEADO','DIGUL (also UWIMBU)',
 'DJERID','DNIEPER','DNIESTR','DOCE','DON','DONG','DONG NAI','DORDOGNE',
 'DOURO (also DUERO)','DRAA','DRAMSELVA','DRYSDALE','DURACK','EASTMAIN',
 'EBRO','EEL','ELBE','ELLICE','ESMERALDAS','ESSEQUIBO','FERGUSON',
 'FEUILLES','FITZROY','FLINDERS','FORTESCUE','FRASER','FUERTE NUEVO',
 'GALANA','GALLEGOS','GAMBIA','GANGES','GARONNE (also RIO GARONA)',
 'GASCOYNE','GEBA (also KAYANGA)','GEORGE','GILBERT','GLOMAA','GOASCORAN',
 'GODAVARI','GONO','GOTA','GOURITZ','GRANDE DE MATAGALPA',
 'GREAT FISH (also GROOT-VIS)','GREAT KEI (also GROOT-KEI)',
 'GREAT SALT LAKE','GREAT SCARCIES (also KOLENTE)','GUADALQUIVIR',
 'GUADIANA','GUAYAS','GUDENA','GUIR','GURUPI','HAI HE','HAMUN',
 'HAMUN-I-LORA','HAMUN-I-MASHKEL','HAN-GANG','HANJIANG XIXI','HAR US NUUR',
 'HARI','HAYES','HINGOL','HODNA','HONDO','HORNADAY','HORTON','HUAI HE',
 'HUASCO','HUDSON','IIJOKI','INCOMATI (also KOMATI)','INDIGIRKA','INDUS',
 'IRRAWADDY','ISHIKARI','ITAPECURU','ITAPICURU','JACUI','JAGUARIBE',
 'JAMES','JEQUITINHONHA','JOKULSA A FJOLLUM','JUBBA (also GENALE WENZ)',
 'JURADO','KALIXALVEN','KAMCHATKA','KAPUAS','KARNAPHULI','KELANTAN','KEM',
 'KEMIJOKI','KHATANGA','KINABATANGAN','KISO','KITAKAMI','KIZILIRMAK',
 'KLAMATH','KOBUK','KOKEMAENJOKI','KOLOK','KOLYMA','KOUILOU','KOVDA',
 'KRISHNA','KUANTAN','KUBAN','KUNENE (also CUNENE)','KURA','KUSKOKWIM',
 'KYMIJOKI','LA GRANDE','LAGARFLJOT','LAGUNA DEL BAYO','LAGUNA ESCONDIDA',
 'LAGUNA MUCAR','LAKE BALKHASH','LAKE EYRE','LAKE GAIRDNER','LAKE NATRON',
 'LAKE TORRENS','LAKE TURKANA','LAKE URMIA','LAM (also XONGKA)','LAUCA',
 'LEICHHARDT','LEMPA','LENA','LIAO HE','LIMA (also LIMIA)','LIMARI',
 'LIMPOPO','LITTLE MECATINA','LITTLE SCARCIES (also KABA)','LOA',
 'LOFFA (also LOFA RIVER)','LOIRE','LORENTZ','LUAN HE','LULEALVEN','LURIO',
 'MA','MAAS (also MEUSE)','MACKENZIE','MAE KLONG','MAGDALENA','MAHAKAM',
 'MAHANADI','MAHI','MAJES','MAMBERAMO','MANGOKY','MANICOUAGAN',
 'MANO (also MANO RIVER)','MAPUTO (also LUSUTFU, also USUTU)',
 'MAR CHIQUITA','MARITSA RIVER','MARONI','MBINI (also WOLEU)','MCARTHUR',
 'MEARIM','MEDJERDA (also MAJARDAH)','MEGHNA','MEKONG','MELRHIR',
 'MERIN (also MIRIM)','MERRIMACK','MESSALO','MEZEN','MIN JIANG',
 'MINO (also MINHO)','MIRA','MISSISSIPPI','MITCHELL','MOA (also MAKONA)',
 'MOBILE RIVER','MOGAMI','MONO','MOOSE','MOTAGUA','MOTALA STROM',
 'MOULOUYA','MUCURI','MURCHISON','MURRAY','NADYM','NAKDONG','NARMADA',
 'NARVA','NASS','NATASHQUAN','NEGRO','NELSON',
 'NEMAN (also NEMUNAS, also NYOMAN)','NESTOS (also MESTA)','NEVA','NIGER',
 'NILE','NOATAK','NOTTAWAY','NTEM','NUECES','NUSHAGAK','NYANGA','NYONG',
 'OB','OCONA','ODER','OGOOUE','OIAPOQUE (also OYAPOK)',
 'OKAVANGO (also CUBANGO)','OLENEK','OLFUSA','OLIFANTS','OMOLOY','ONEGA',
 'ORANGE','ORD','ORINOCO','ORONTES','OUEME','OULUJOKI','PAHANG','PAKCHAN',
 'PALENA (also CORCOVADO)','PALYAVAAM','PANGANI','PANUCO','PAPALOAPAN',
 'PARAGUACU','PARAIBA','PARAIBA DO SUL','PARANA','PARNAIBA','PASCUA',
 'PATIA','PATUCA','PAZ','PEARL','PECHORA','PEE DEE','PENNER','PENOBSCOT',
 'PERAK','PINDARE','PIVDENNY BUH (also SOUTHERN BUG)','PO','PONOY','POOPO',
 'POPIGAY','POTOMAC','PRA','PUELO','PULAU (also EILANDEN)',
 'PUNGWE  (also PUNGOE)','PUR','PURARI','PYASINA','QIANTANG JIANG',
 'QUOICH','RAJANG','RAPEL','RED','RHARSA','RHINE','RHONE',
 'RIBEIRA DO IGUAPE','RIO DE LA PLATA','RIO GRANDE (also BRAVO)','ROANOKE',
 'ROGUE','ROPER','RUFIJI','RUKWA','RUPERT','RUVU','RUVUMA (also ROVUMA)',
 'SABINE','SACRAMENTO','SAGUENAY','SAINT CROIX',
 'SAINT JOHN (also SAINT-JEAN)','SAINT JOHN RIVER','SAINT JOHNS',
 'SAINT LAWRENCE (also SAINT-LAURENT)','SAINT PAUL RIVER','SAKARYA',
 'SALAR DEL PULAR','SALINA DE JAMA','SALINAS','SALSA','SALWEEN',
 'SAN ANTONIO','SAN JOAQUIN','SAN JUAN','SAN PEDRO','SANAGA','SANTA',
 'SANTA CRUZ','SANTEE','SANTIAGO','SAO FRANCISCO','SARSTOON','SARYSU',
 'SASSANDRA','SAVANNAH','SAVE','SCHELDE (also ESCAUT)','SEAL','SEBOU',
 'SEINE','SEMBAKUNG','SENEGAL','SEPIK','SERRANO','SEVERN',
 'SEVERNAYA DVINA','SHANNON','SHATT AL ARAB','SHINANO GAWA','SITTANG',
 'SIXAOLA','SKAGIT','SKEENA','SKJERN A','SOUTH ESK','SPEY','STIKINE',
 'STRUMA RIVER','SUCHIATE','SUNGAI FLY (also FLY RIVER)','SURINAME RIVIER',
 'SUSITNA','SUSQUEHANNA','SUWANNEE','SVARTA','TAFNA','TAGUS','TAKU',
 'TALAS','TANA','TANO (also TANOE)','TAPI','TARIM HE (also TALIMU HE)',
 'TAZ','TENRYU','THAMES','THELON','THJORSA','THLEWIAZA','THU BON',
 'TIJUANA (also TIA JUANA)','TITICACA','TOCANTINS','TONE','TORGHAY',
 'TORNEALVEN (also TORNIONJOKI, also TORNIONVAYLA)','TRENT','TRINITY',
 'TSIRIBIHINA','TUBAN','TULOMA','TUMBES (also TUMBEZ)','TUMEN','TWEED',
 'ULUA','ULUNGUR HU','UMBA','UMBELUZI (also UMBULUZI)','URAL (also ORAL)',
 'URUGUAY','USUMACINTA','UTAMBONI (also MITEMBONI)','UTHUKELA',
 'UVS NUUR (also UBSU-NUR)','VALDIVIA','VAM CO','VARDAR','VARZUGA',
 'VASA BARRIS','VERDE','VICTORIA','VISTULA','VOLGA','VOLTA','VYG','WAIAU',
 'WAIKATO','WESER','WINISK','YALU RIVER','YANA','YANGTZE','YAQUI','YELCHO',
 'YELLOW RIVER','YENISEY','YODO','YSYK-KOL (also ISSYK-KUL)','YUKON',
 'ZAMBEZI','ZAPALERI','ZARUMILLA','ZHEM (also EMBA)','ZHU']


In [18]:
plt.ioff()

bName=[];v1=[];v2=[];vcc=[];gFill=[];lens=[];
Te=[];eVal=[];eDoF=[];sig=[];mxV=[];mxVloc=[]; lags=[]
for detrend in detrends:
    for gapFill in gapFills:
        for variable in variables:
            for basin in basins:
                for variable2 in variable2s:
                    #read basin dataframe
                    path = './Basin_data/'+variable+'_'+basin+'.csv'
                    path2 = './Basin_data/'+variable2+'_'+basin+'.csv'
                    glwsPath = './Basin_data/glws2_tws_anom_mm_'+basin+'.csv'

                    #Read in trends and detrend data
                    bDF = pd.read_csv(path)
                    bDF = bDF.loc[(bDF.Year>=2002)&(bDF.Year<=2022),:]
                    if (np.sum(np.isfinite(bDF.iloc[:,1:].values))==0):continue
                    vals = bDF.iloc[:,1:].values.ravel()

                    glwsDF = pd.read_csv(glwsPath)
                    glwsDF = glwsDF.loc[(glwsDF.Year>=2002)&(glwsDF.Year<=2022),:]
                    if (np.sum(np.isfinite(glwsDF.iloc[:,1:].values))==0):continue
                    vals = glwsDF.iloc[:,1:].values.ravel()
                    if detrend=='linear':
                        index = sm.add_constant(np.array(range(vals.size)))
                        b = sm.OLS(vals[np.isfinite(vals)],index[np.isfinite(vals)]).fit().params[0]
                        m = sm.OLS(vals[np.isfinite(vals)],index[np.isfinite(vals)]).fit().params[1]
                        dtrnd = vals - (m*index[:,1]+b) #detrend
                        plt.figure() #plot
                        plt.plot(glwsDF.iloc[:,1:].values.ravel())
                        plt.plot((m*index[:,1]+b))
                        plt.title(variable+' '+basin)
                        plt.savefig('./Figures/trends/glws_'+variable+'_'+basin+'_'+detrend+'.png')
                        plt.close()
                    elif detrend[:5]=='Gauss':
                        gSig = int(detrend[-2:])
                        #To account for a missing values in locations we can drop and re-weight the remaining values by counting non-zero values        
                        iVal = np.copy(vals)
                        valCount = np.copy(vals)
                        valCount[~np.isnan(vals)]=1
                        valCount[np.isnan(valCount)]=0
                        iVal[np.isnan(iVal)]=0
                        #divide by the count to re-weight based on the # of values going into each smoothing filter
                        # a kernel radius of 36 months is about 3-years, yielding something like a 10-year running mean
                        GauExp = ndimage.gaussian_filter1d(iVal,sigma=gSig)/ndimage.gaussian_filter1d(valCount,sigma=gSig)
                        dtrnd = GauExp - vals
                        plt.figure() #plot
                        plt.plot(glwsDF.iloc[:,1:].values.ravel())
                        plt.plot(GauExp)
                        plt.title(variable+' '+basin)
                        plt.savefig('./Figures/trends/glws_'+variable+'_'+basin+'_'+detrend+'.png')
                        plt.close()

                    glwsDF.iloc[:,1:] = dtrnd.reshape([dtrnd.size//12,12]) #insert detrended data back to the df



In [21]:
plt.ioff()

bName=[];v1=[];v2=[];vcc=[];gFill=[];lens=[];
Te=[];eVal=[];eDoF=[];sig=[];mxV=[];mxVloc=[]; lags=[]
for detrend in detrends:
    for gapFill in gapFills:
        for variable in variables:
            for basin in basins:
                for variable2 in variable2s:
                    #read basin dataframe
                    path = './Basin_data/'+variable+'_'+basin+'.csv'
                    path2 = './Basin_data/'+variable2+'_'+basin+'.csv'
                    glwsPath = './Basin_data/glws2_tws_anom_mm_'+basin+'.csv'

                    #Read in trends and detrend data
                    bDF = pd.read_csv(path)
                    bDF = bDF.loc[(bDF.Year>=2002)&(bDF.Year<=2022),:]
                    if (np.sum(np.isfinite(bDF.iloc[:,1:].values))==0):continue
                    vals = bDF.iloc[:,1:].values.ravel()

                    if detrend=='linear':
                        index = sm.add_constant(np.array(range(vals.size)))
                        b = sm.OLS(vals[np.isfinite(vals)],index[np.isfinite(vals)]).fit().params[0]
                        m = sm.OLS(vals[np.isfinite(vals)],index[np.isfinite(vals)]).fit().params[1]
                        plt.figure() #plot
                        plt.plot(bDF.iloc[:,1:].values.ravel())
                        plt.plot(index[:,1],m*index[:,1]+b)
                        plt.title(variable+' '+basin)
                        plt.savefig('./Figures/trends/'+variable+'_'+basin+'_'+detrend+'.png')
                        plt.close()
                        dtrnd = vals - (m*index[:,1]+b) #detrend
                    elif detrend[:5]=='Gauss':
                        gSig = int(detrend[-2:])
                        #To account for a missing values in locations we can drop and re-weight the remaining values by counting non-zero values        
                        iVal = np.copy(vals)
                        valCount = np.copy(vals)
                        valCount[~np.isnan(vals)]=1
                        valCount[np.isnan(valCount)]=0
                        iVal[np.isnan(iVal)]=0
                        #divide by the count to re-weight based on the # of values going into each smoothing filter
                        # a kernel radius of 36 months is about 3-years, yielding something like a 10-year running mean
                        GauExp = ndimage.gaussian_filter1d(iVal,sigma=gSig)/ndimage.gaussian_filter1d(valCount,sigma=gSig)
                        plt.figure() #plot
                        plt.plot(bDF.iloc[:,1:].values.ravel())
                        plt.plot(GauExp)
                        plt.title(variable+' '+basin)
                        plt.savefig('./Figures/trends/'+variable+'_'+basin+'_'+detrend+'.png')
                        plt.close()
                        dtrnd = GauExp - vals
                    else:
                        print('not a valid detrending method')
                        break
                    bDF.iloc[:,1:] = dtrnd.reshape([dtrnd.size//12,12]) #insert detrended data back to the df

                    bDF2 = pd.read_csv(path2)
                    bDF2 = bDF2.loc[(bDF2.Year>=2002)&(bDF2.Year<=2022),:]
                    if (np.sum(np.isfinite(bDF2.iloc[:,1:].values))==0):continue
                    vals = bDF2.iloc[:,1:].values.ravel()
                    if detrend=='linear':
                        index = sm.add_constant(np.array(range(vals.size)))
                        b = sm.OLS(vals[np.isfinite(vals)],index[np.isfinite(vals)]).fit().params[0]
                        m = sm.OLS(vals[np.isfinite(vals)],index[np.isfinite(vals)]).fit().params[1]
                        plt.figure() #plot
                        plt.plot(bDF2.iloc[:,1:].values.ravel())
                        plt.plot(index[:,1],m*index[:,1]+b)
                        plt.title(variable2+' '+basin)
                        plt.savefig('./Figures/trends/'+variable2+'_'+basin+'_'+detrend+'.png')
                        plt.close()
                        dtrnd = vals - (m*index[:,1]+b) #detrend
                    elif detrend[:5]=='Gauss':
                        gSig = int(detrend[-2:])
                        #To account for a missing values in locations we can drop and re-weight the remaining values by counting non-zero values        
                        iVal = np.copy(vals)
                        valCount = np.copy(vals)
                        valCount[~np.isnan(vals)]=1
                        valCount[np.isnan(valCount)]=0
                        iVal[np.isnan(iVal)]=0
                        #divide by the count to re-weight based on the # of values going into each smoothing filter
                        # a kernel radius of 36 months is about 3-years, yielding something like a 10-year running mean
                        GauExp = ndimage.gaussian_filter1d(iVal,sigma=gSig)/ndimage.gaussian_filter1d(valCount,sigma=gSig)
                        plt.figure() #plot
                        plt.plot(bDF2.iloc[:,1:].values.ravel())
                        plt.plot(GauExp)
                        plt.title(variable2+' '+basin)
                        plt.savefig('./Figures/trends/'+variable2+'_'+basin+'_'+detrend+'.png')
                        plt.close()
                        dtrnd = GauExp - vals
                    bDF2.iloc[:,1:] = dtrnd.reshape([dtrnd.size//12,12]) #insert detrended data back to the df

                    glwsDF = pd.read_csv(glwsPath)
                    glwsDF = glwsDF.loc[(glwsDF.Year>=2002)&(glwsDF.Year<=2022),:]
                    if (np.sum(np.isfinite(glwsDF.iloc[:,1:].values))==0):continue
                    vals = glwsDF.iloc[:,1:].values.ravel()
                    if detrend=='linear':
                        index = sm.add_constant(np.array(range(vals.size)))
                        b = sm.OLS(vals[np.isfinite(vals)],index[np.isfinite(vals)]).fit().params[0]
                        m = sm.OLS(vals[np.isfinite(vals)],index[np.isfinite(vals)]).fit().params[1]
                        dtrnd = vals - (m*index[:,1]+b) #detrend
                    elif detrend[:5]=='Gauss':
                        gSig = int(detrend[-2:])
                        #To account for a missing values in locations we can drop and re-weight the remaining values by counting non-zero values        
                        iVal = np.copy(vals)
                        valCount = np.copy(vals)
                        valCount[~np.isnan(vals)]=1
                        valCount[np.isnan(valCount)]=0
                        iVal[np.isnan(iVal)]=0
                        #divide by the count to re-weight based on the # of values going into each smoothing filter
                        # a kernel radius of 36 months is about 3-years, yielding something like a 10-year running mean
                        GauExp = ndimage.gaussian_filter1d(iVal,sigma=gSig)/ndimage.gaussian_filter1d(valCount,sigma=gSig)
                        plt.figure() #plot
                        plt.plot(glwsDF.iloc[:,1:].values.ravel())
                        plt.plot(GauExp)
                        plt.title(variable+' '+basin)
                        plt.savefig('./Figures/trends/glws_'+variable+'_'+basin+'_'+detrend+'.png')
                        plt.close()
                        dtrnd = GauExp - vals
                    glwsDF.iloc[:,1:] = dtrnd.reshape([dtrnd.size//12,12]) #insert detrended data back to the df

                    #plot climatology
                    plt.figure()
                    bDF.iloc[:,1:].mean(0).plot(title='Climatology of '+variable+'\nin '+basin+' basin')
                    plt.savefig('./Figures/Climatologies/'+variable+'_'+basin+'_'+gapFill+'.png')
                    plt.close()

                    bDFclim = bDF.iloc[:,1:].mean(0)
                    bDFclim = bDFclim.reset_index().rename(columns={'index':'Month',0:'value'})
                    bDF2clim = bDF2.iloc[:,1:].mean(0)
                    bDF2clim = bDF2clim.reset_index().rename(columns={'index':'Month',0:'value'})

                    glwsDFclim = glwsDF.iloc[:,1:].mean(0)
                    glwsDFclim = glwsDFclim.reset_index().rename(columns={'index':'Month',0:'value'})

                    #Calculate anomalies by remove monthly climatology from each month
                    bDF.iloc[:,1:] = bDF.iloc[:,1:] - bDF.iloc[:,1:].mean(0)
                    bDF2.iloc[:,1:] = bDF2.iloc[:,1:] - bDF2.iloc[:,1:].mean(0)
                    glwsDF.iloc[:,1:] = glwsDF.iloc[:,1:] - glwsDF.iloc[:,1:].mean(0)
                    #reshape the data
                    bDF = bDF.melt(id_vars='Year',var_name='Month')
                    bDF2 = bDF2.melt(id_vars='Year',var_name='Month')
                    glwsDF = glwsDF.melt(id_vars='Year',var_name='Month')
                    #create a datetime index
                    dtInd = pd.to_datetime(bDF.Year.astype(int).astype(str) + bDF['Month'], format='%Y%b')
                    bDF['date'] = dtInd
                    dtInd2 = pd.to_datetime(bDF2.Year.astype(int).astype(str) + bDF2['Month'], format='%Y%b')
                    bDF2['date'] = dtInd2
                    dtIndglws = pd.to_datetime(glwsDF.Year.astype(int).astype(str) + glwsDF['Month'], format='%Y%b')
                    glwsDF['date'] = dtIndglws          
                    bDF=bDF.set_index('date')
                    bDF2=bDF2.set_index('date')
                    glwsDF=glwsDF.set_index('date')

                    if (variable=='grun_ens_runoff_mm_day')&(variable2=='grun_ens_runoff_mm_day'):
                        dtInd = dtInd[(dtInd.dt.year>1981)] #limit to satellite era to be more similar to other vars   
                        dtInd2 = dtInd2[(dtInd2.dt.year>1981)]
                    if variable=='grace_lwe_cm':
                        if gapFill == 'clim':
                            #replace missing values with climatology (which is = 0 in anomaly space)
                            bDF.loc[np.isnan(bDF['value']),'value'] = 0 
                        elif gapFill == 'glws2':
                            dt_t = np.isin(dtInd,dtIndglws)
                            bDF.loc[(np.isnan(bDF['value']))&(list(dt_t)),'value'] = \
                                glwsDF.loc[(np.isnan(bDF['value']))&(list(dt_t)),'value'].values/10 #convert to cm
                            dtInd = dtInd[(dtInd.dt.year>2002)]
                        else:
                            print('not a valid gap fill option')
                            break
                        dtInd = dtInd[(dtInd.dt.year<=2022)]
                    if variable2=='grace_lwe_cm':
                        if gapFill == 'clim':
                            #replace missing values with climatology (which is = 0 in anomaly space)
                            bDF2.loc[np.isnan(bDF2['value']),'value'] = 0
                        elif gapFill == 'glws2':
                            dt_t = np.isin(dtInd2,dtIndglws)
                            bDF2.loc[(np.isnan(bDF2['value']))&(list(dt_t)),'value'] = \
                                glwsDF.loc[(np.isnan(bDF2['value']))&(list(dt_t)),'value'].values/10 #convert to cm
                            dtInd2 = dtInd2[(dtInd2.dt.year>2002)]#glws v2 not available in 2002
                        else:
                            print('not a valid gap fill option')
                            break
                        dtInd = dtInd[(dtInd.dt.year<=2022)]

                    #choose only values that exist in both datasets to calculate the cross correlation
                    dts = set(dtInd).intersection(dtInd2)
                    bDF=bDF.loc[list(dts)]
                    bDF2=bDF2.loc[list(dts)]

                    #reorder the data chronologically
                    bDF = bDF.sort_values(by='date')
                    bDF2 = bDF2.sort_values(by='date')

                    #Calculate the autocorrelation
                    if variable == variable2:
                        anoms = bDF.value.values
                        anoms = anoms-np.nanmean(anoms)
                        std = np.std(anoms)
                        corr = sm.tsa.stattools.ccf(anoms/std,anoms/std,adjusted=True,fft=False)
                        #plt.figure()
                        #plt.plot(corr[:9])
                        #plt.title(variable+' autocorrelation out to 9 months')
                        #plt.savefig('./Figures/crosscorrelations_individual_vars/'+variable+'_'+basin+'.png')
                        #plt.close()
                    else:
                        anoms = bDF.value.values
                        anoms2 = bDF2.value.values
                        anoms = anoms-np.nanmean(anoms)
                        anoms2 = anoms2-np.nanmean(anoms2)
                        std = np.std(anoms)
                        std2 = np.std(anoms2)
                        corr = sm.tsa.stattools.ccf(anoms/std,anoms2/std2,adjusted=True,fft=False)
                        #plt.figure()
                        #plt.plot(corr[:9])
                        #plt.title(variable+' lags \n'+variable2+'\ncross-correlation out to 9 months')
                        #plt.savefig('./Figures/crosscorrelations_individual_vars/'+variable+'_'+variable2+'_'+basin+'.png')
                        #plt.close()

                    sig90 = scipy.stats.norm.ppf((1 + 0.90)/2)/np.sqrt(len(anoms))
                    #sig90 = scipy.stats.norm.ppf(0.90, loc=np.mean(corr[:48]), scale=np.std(corr[:48]))

                    Te_t,eVal_t,eDoF_t, mxV_t, mxVloc_t = e_fold(corr,len(anoms))

                    lags = np.append(lags, np.arange(1,25,1))
                    sig = np.append(sig,1*(sig90<=corr[:24]),0)
                    vcc = np.append(vcc,corr[:24],0)
                    v1 = np.append(v1,np.repeat(variable,24),0)
                    v2 = np.append(v2,np.repeat(variable2,24),0)
                    bName = np.append(bName,np.repeat(basin,24),0)
                    gFill = np.append(gFill,np.repeat(gapFill,24),0)
                    lens = np.append(lens,np.repeat(len(anoms),24),0)
                    Te = np.append(Te, np.repeat(Te_t,24),0)
                    eVal = np.append(eVal, np.repeat(eVal_t,24),0)
                    eDoF = np.append(eDoF, np.repeat(eDoF_t,24),0)
                    mxV = np.append(mxV, np.repeat(mxV_t,24),0)
                    mxVloc = np.append(mxVloc, np.repeat(mxVloc_t,24),0)

            df =pd.DataFrame(data={'basin':bName,'var1':v1,'var2':v2,'cross_corr':vcc,'gapFill':gFill,'detrend':detrend,'lag':lags,
                                   'n':lens,'Te':Te,'eDoF':eDoF,'90pctSig':sig,'maxCorr':mxV,'maxCorrLag':mxVloc})
            path = './crosscorr_dfs/basin_'+gapFill+'_'+detrend+'_'+variable+'.pkl'
            df.to_pickle(path)

In [22]:
plt.ioff()

colors = {
'e_gleam_mm_mon':'limegreen',
'glws2_gwater_mm':'navy',
'grace_lwe_cm':'red',
'grun_ens_runoff_mm_day':'blue',
'prec_fldas_mm_day':'cornflowerblue',
'smroot_gleam_mm3_mm3':'saddlebrown',
'smsurf_gleam_mm3_mm3':'peru',
'lai_gimms4g_m2_m2':'green'
}

Te=[]
for detrend in detrends:
    for igapFill in gapFills:
        for variable in variables:
            path = './crosscorr_dfs/basin_'+igapFill+'_'+detrend+'_'+variable+'.pkl'
            df=pd.read_pickle(path)
            basins = list(df.basin.unique())
            for basin in basins:
                plt.figure()
                for variable2 in variable2s:
                    if variable==variable2:continue
                    corrTS = df[(df.gapFill==igapFill)&(df.basin==basin)&(df.var1==variable)&(df.var2==variable2)].cross_corr.values
                    sigCorr = df.loc[(df.gapFill==igapFill)&(df.basin==basin)&(df.var1==variable)&(df.var2==variable2),'90pctSig'].values
                    plt.plot(corrTS,label=str(variable2),color=colors[variable2])
                    if np.sum(sigCorr>0):
                        plt.scatter(np.where(sigCorr==1)[0],corrTS[sigCorr==1],color=colors[variable2])
                plt.legend()
                plt.title(basin+'\n'+variable+' lags [X]')
                plt.savefig('./Figures/crosscorrelations_all_vars/'+variable+'_allVars_'+basin+'_'+igapFill+'_'+detrend+'.png')
                plt.close()

KeyboardInterrupt: 

In [19]:
colors = {
'JUBBA (also GENALE WENZ)':'red',
'NILE':'cornflowerblue',
'ORANGE':'peru',
'LIMPOPO':'green',
'MEKONG':'orange',
'ARAL SEA':'darkgreen',
'INDUS':'firebrick',
'COLORADO (also COLORADO RIVER)':'indianred',
'PARANA':'blue',
'SHATT AL ARAB':'black',
'NIGER':'grey',
'MURRAY':'navy',
}

    
for detrend in detrends:
    for igapFill in gapFills:
        for variable in variables:
            fig = plt.figure()
            path = './crosscorr_dfs/basin_'+igapFill+'_'+detrend+'_'+variable+'.pkl'
            df=pd.read_pickle(path)
            basins = colors.keys()
            for basin in basins:
                corrTS = df[(df.gapFill==igapFill)&(df.basin==basin)&(df.var1==variable)&(df.var2==variable)].cross_corr.values
                sigCorr = df.loc[(df.gapFill==igapFill)&(df.basin==basin)&(df.var1==variable)&(df.var2==variable),'90pctSig'].values
                plt.plot(corrTS,label=str(basin),color=colors[basin])
                if np.sum(sigCorr>0):
                        plt.scatter(np.where(sigCorr==1)[0],corrTS[sigCorr==1],color=colors[basin])
                plt.legend()
            fig.set_size_inches(10,8)
            plt.title(variable+' autocorrelation')
            plt.savefig('./Figures/autocorrelations_all_basins/'+variable+'_'+igapFill+'_'+detrend+'_allbasins.png')
            plt.close()

FileNotFoundError: [Errno 2] No such file or directory: './crosscorr_dfs/basin_glws2_Gauss_smroot_gleam_mm3_mm3.pkl'

In [7]:
for detrend in detrends:
    for igapFill in gapFills:
        for variable in variables:
            path = './crosscorr_dfs/basin_'+igapFill+'_'+detrend+'_'+variable+'.pkl'
            df=pd.read_pickle(path)
            basins = colors.keys()
            df = df.set_index('basin').loc[list(basins)].reset_index()

            plt.figure()
            eFold = df.loc[(df.gapFill==igapFill)&(df.var1==variable)&(df.var2==variable),['basin','Te']]
            eFold.drop_duplicates().plot.bar(x='basin',y='Te')
            plt.title(variable+' autocorrelation e-folding time (months)')
            plt.tight_layout()
            plt.savefig('./Figures/e_Folding/eFolding_autocorrelation_'+variable+'_'+igapFill+'_'+detrend+'_allbasins.png')      
            plt.close()

In [29]:
import geopandas as gpd
import cartopy
import matplotlib as mpl
from matplotlib.colors import Normalize
import matplotlib.cm as cm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
projection = ccrs.Robinson(central_longitude=0)#ccrs.PlateCarree()
largeReg = [-180.5, 180.5, -45, 65]

cbarMax = 15

for detrend in detrends:
    path = './crosscorr_dfs/basin_'+igapFill+'_'+detrend+'_'+variable+'.pkl'
    df=pd.read_pickle(path)

    bdf = gpd.read_file('/Volumes/Data_Archive/Data/basins/mrb_shp_zip/mrb_basins.shp').to_crs(crs=projection)
    bdf = pd.merge(bdf, df, left_on='RIVER_BASI',right_on='basin')
    dfTe = bdf[['basin','Te','var1','var2','geometry']].drop_duplicates()
    dfTe = dfTe.loc[dfTe.var2=='grace_lwe_cm',:]
    dfmxC = bdf[['basin','maxCorr','var1','var2','geometry']].drop_duplicates()
    dfmxC = dfmxC.loc[dfmxC.var2=='grace_lwe_cm',:]
    dfmxL = bdf[['basin','maxCorrLag','var1','var2','geometry']].drop_duplicates()
    dfmxL = dfmxL.loc[dfmxL.var2=='grace_lwe_cm',:]

    reader = shpreader.Reader('/Volumes/Data_Archive/Data/adminBoundaries/ne_50m_admin_0_countries/ne_50m_admin_0_countries.shp')
    worldShps = list(reader.geometries())
    ADM0 = cfeature.ShapelyFeature(worldShps, ccrs.PlateCarree())
    
    fig = plt.figure(1)
    ccrs.PlateCarree()
    ax1 = plt.subplot(111,projection=projection);
    plt.title("GRACE LWE Te (months)")
    ax1.coastlines(resolution='50m',zorder=3.5);
    ax1.add_feature(ADM0, facecolor='grey', edgecolor='k',linewidth=0.5,zorder=0.5)
    ax1.add_feature(cartopy.feature.BORDERS,zorder=3.5)  
    ax1.set_extent(largeReg,crs=ccrs.PlateCarree(central_longitude=0.0))
    ax1.add_feature(cfeature.OCEAN, zorder=10, edgecolor='k',facecolor='aliceblue')
    dfTe.plot('Te',ax=ax1,cmap='YlGnBu',zorder=1,vmax=cbarMax,vmin=0)
    cbar_ax1 = fig.add_axes([0.17,0.1,0.7,0.05])
    norm = Normalize(vmin=0, vmax=cbarMax, clip=False)
    cb1 = mpl.colorbar.ColorbarBase(cbar_ax1, orientation='horizontal', 
                                    cmap=cm.YlGnBu, norm=norm, extend='both')
    fig.set_size_inches(10,5)
    plt.savefig('./Figures/maps/autocorr/autocorrelation_'+variable+'_Te_'+igapFill+'_'+detrend+'_allbasins.png')  
    plt.close()

In [ ]:
for detrend in detrends:
    path = './crosscorr_dfs/basin_'+igapFill+'_'+detrend+'_'+variable+'.pkl'
    df=pd.read_pickle(path)

    bdf = gpd.read_file('/Volumes/Data_Archive/Data/basins/mrb_shp_zip/mrb_basins.shp').to_crs(crs=projection)
    bdf = pd.merge(bdf, df, left_on='RIVER_BASI',right_on='basin')
    dfTe = bdf[['basin','Te','var1','var2','geometry']].drop_duplicates()
    dfTe = dfTe.loc[dfTe.var2=='grace_lwe_cm',:]
    dfmxC = bdf[['basin','maxCorr','var1','var2','geometry']].drop_duplicates()
    dfmxC = dfmxC.loc[dfmxC.var2=='grace_lwe_cm',:]
    dfmxL = bdf[['basin','maxCorrLag','var1','var2','geometry']].drop_duplicates()
    dfmxL = dfmxL.loc[dfmxL.var2=='grace_lwe_cm',:]
    for var2 in variable2s:
        dfmxC = bdf.loc[bdf['90pctSig']==1,:]
        dfmxC = dfmxC[['basin','cross_corr','lag','var1','var2','geometry']].drop_duplicates()
        dfmxC = dfmxC.loc[dfmxC.var2==var2,:]
        idx = dfmxC.groupby(['basin','var1','var2'])['lag'].idxmax()
        max_lag = dfmxC.loc[idx]
        fig = plt.figure(1)
        ccrs.PlateCarree()
        ax1 = plt.subplot(121,projection=projection);
        plt.title(var2+'\n correlation (at last sig)')
        ax1.coastlines(resolution='50m',zorder=3.5);
        ax1.add_feature(ADM0, facecolor='grey', edgecolor='k',linewidth=0.5,zorder=0.5)
        ax1.add_feature(cartopy.feature.BORDERS,zorder=3.5)  
        ax1.set_extent(largeReg,crs=ccrs.PlateCarree(central_longitude=0.0))
        ax1.add_feature(cfeature.OCEAN, zorder=10, edgecolor='k',facecolor='aliceblue')
        max_lag.plot('cross_corr',ax=ax1,cmap='PiYG',zorder=1,vmin=-.7,vmax=.7)

        cbar_ax1 = fig.add_axes([0.13,0.1,0.35,0.05])
        norm = Normalize(vmin=-.7, vmax=.7, clip=False)
        cb1 = mpl.colorbar.ColorbarBase(cbar_ax1, orientation='horizontal', 
                                        cmap=cm.PiYG, norm=norm, extend='both')

        ax2 = plt.subplot(122,projection=projection);
        plt.title(var2+'\n last sig corr lag')
        ax2.coastlines(resolution='50m',zorder=3.5);
        ax2.add_feature(ADM0, facecolor='grey', edgecolor='k',linewidth=0.5,zorder=0.5)
        ax2.add_feature(cartopy.feature.BORDERS,zorder=3.5)  
        ax2.set_extent(largeReg,crs=ccrs.PlateCarree(central_longitude=0.0))
        ax2.add_feature(cfeature.OCEAN, zorder=10, edgecolor='k',facecolor='aliceblue')
        max_lag.plot('lag',ax=ax2,cmap='YlGnBu',zorder=1,vmin=0,vmax=24)

        cbar_ax2 = fig.add_axes([0.56,0.1,0.35,0.05])
        norm = Normalize(vmin=0, vmax=24, clip=False)
        cb2 = mpl.colorbar.ColorbarBase(cbar_ax2, orientation='horizontal', 
                                        cmap=cm.YlGnBu, norm=norm, extend='both')

        fig.set_size_inches(20,5)
        plt.savefig('./Figures/maps/crosscorr/sig_maxCrossCorr_'+variable+'_'+var2+'_'+igapFill+'_'+detrend+'_allbasins.png')  
        plt.close()

In [ ]:
for detrend in detrends:
    path = './crosscorr_dfs/basin_'+igapFill+'_'+detrend+'_'+variable+'.pkl'
    df=pd.read_pickle(path)

    bdf = gpd.read_file('/Volumes/Data_Archive/Data/basins/mrb_shp_zip/mrb_basins.shp').to_crs(crs=projection)
    bdf = pd.merge(bdf, df, left_on='RIVER_BASI',right_on='basin')
    dfTe = bdf[['basin','Te','var1','var2','geometry']].drop_duplicates()
    dfTe = dfTe.loc[dfTe.var2=='grace_lwe_cm',:]
    dfmxC = bdf[['basin','maxCorr','var1','var2','geometry']].drop_duplicates()
    dfmxC = dfmxC.loc[dfmxC.var2=='grace_lwe_cm',:]
    dfmxL = bdf[['basin','maxCorrLag','var1','var2','geometry']].drop_duplicates()
    dfmxL = dfmxL.loc[dfmxL.var2=='grace_lwe_cm',:]
    for var2 in variable2s:
        dfmxC = bdf[['basin','maxCorr','maxCorrLag','var1','var2','geometry']].drop_duplicates()
        dfmxC = dfmxC.loc[dfmxC.var2==var2,:]
        fig = plt.figure(1)
        ccrs.PlateCarree()
        ax1 = plt.subplot(121,projection=projection);
        plt.title(var2+'\n max correlation')
        ax1.coastlines(resolution='50m',zorder=3.5);
        ax1.add_feature(ADM0, facecolor='grey', edgecolor='k',linewidth=0.5,zorder=0.5)
        ax1.add_feature(cartopy.feature.BORDERS,zorder=3.5)  
        ax1.set_extent(largeReg,crs=ccrs.PlateCarree(central_longitude=0.0))
        ax1.add_feature(cfeature.OCEAN, zorder=10, edgecolor='k',facecolor='aliceblue')
        dfmxC.plot('maxCorr',ax=ax1,cmap='PiYG',zorder=1,vmin=-.7,vmax=.7)

        cbar_ax1 = fig.add_axes([0.13,0.1,0.35,0.05])
        norm = Normalize(vmin=-.7, vmax=.7, clip=False)
        cb1 = mpl.colorbar.ColorbarBase(cbar_ax1, orientation='horizontal', 
                                        cmap=cm.PiYG, norm=norm, extend='both')

        ax2 = plt.subplot(122,projection=projection);
        plt.title(var2+'\n max correlation lag')
        ax2.coastlines(resolution='50m',zorder=3.5);
        ax2.add_feature(ADM0, facecolor='grey', edgecolor='k',linewidth=0.5,zorder=0.5)
        ax2.add_feature(cartopy.feature.BORDERS,zorder=3.5)  
        ax2.set_extent(largeReg,crs=ccrs.PlateCarree(central_longitude=0.0))
        ax2.add_feature(cfeature.OCEAN, zorder=10, edgecolor='k',facecolor='aliceblue')
        dfmxC.plot('maxCorrLag',ax=ax2,cmap='YlGnBu',zorder=1,vmin=0,vmax=12)

        cbar_ax2 = fig.add_axes([0.56,0.1,0.35,0.05])
        norm = Normalize(vmin=0, vmax=12, clip=False)
        cb2 = mpl.colorbar.ColorbarBase(cbar_ax2, orientation='horizontal', 
                                        cmap=cm.YlGnBu, norm=norm, extend='both')

        fig.set_size_inches(20,5)
        plt.savefig('./Figures/maps/crosscorr/maxCrossCorr_'+variable+'_'+var2+'_'+igapFill+'_'+detrend+'_allbasins.png')  
        plt.close()